# Data Pre-Processing

Unlike other students, my dataset was not pulled from an existing dataset. Instead I utilized various scripts that I wrote to collect and process the data from an existing database

The database I used to pull match data was called opendota. I used their database explorer and SQl in combination to pull the raw match data that I would be using in my formed dataset. 

This gave me a dataset that I could process. Because the pulled data only contained identifying marks of the selected matches that I actually wanted data on. I then had to process those matches utilizing their api, in order to pull the needed features from the match data.

In [ ]:
import requests
import json
import pandas as pd
import time

def _call(url):
    re = requests.get(url)
    load_re = json.loads(re.text)
    return (load_re)


def get_match(match_id):
    time.sleep(.055)
    url = "https://api.opendota.com/api/matches/" + str(match_id) + '?api_key=#######-####-####-####-############' #my api key
    return (_call(url))

def get_player_info_set(matches):
    data_list = []
    df = pd.DataFrame(
        columns=['match_id', 'radiant_win', 'player_1', 'player_2', 'player_3', 'player_4', 'player_5', 'player_6',
                 'player_7', 'player_8', 'player_9', 'player_10'])
    n = 0
    for i in matches['match_id']:
        print(n)
        data_list.append(i)
        data = get_match(int(i))
        data_list.append(data['radiant_win'])
        for players in data['players']:
            data_list.append(players['hero_id'])
        s = pd.Series(data_list, index=df.columns)
        df = df.append(s, ignore_index=True)
        data_list = []
        n = n + 1
    return (df)

match_data = pd.read_csv('Data\Dataset full.csv')
sample = get_player_info_set(match_data)
export_csv = sample.to_csv(r'D:\Python\Projects\Dota Report\Data\export.csv', index=None, header=True)

Because the dataset contains 20000 matches, the processing function will take some time to process all the data. In addition, due to api limitations, a sleep fucntion had to be included in order to not go over the maximum alloted requests per minute

This gave us a large CSV with all the data and features that I would be working with.

In [16]:
import pandas as pd
df = pd.read_csv(r'C:/Users/caleb/Python/Dota Research Project/Data/outladnersexport.csv')
df.head()

,match_id,radiant_win,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10
0,5173127509,False,11,67,14,128,47,26,76,36,35,84
1,5173127511,False,74,84,21,41,83,35,93,47,128,45
2,5173127513,False,55,35,31,39,47,20,66,26,6,129
3,5173127515,True,4,47,86,5,99,111,16,121,82,93
4,5173127519,True,32,8,22,50,63,74,14,69,105,41


This Data couldn't be processed with my model however. I would be using a logistical regression and additial processing had to be done.

In [17]:
df['Radiant']='R'+df['player_1'].map(str)+','+'R'+df['player_2'].map(str)+','+'R'+df['player_3'].map(str)+','+'R'+df['player_4'].map(str)+','+'R'+df['player_5'].map(str)
df['Dire']='D'+df['player_6'].map(str)+','+'D'+df['player_7'].map(str)+','+'D'+df['player_8'].map(str)+','+'D'+df['player_9'].map(str)+','+'D'+df['player_10'].map(str)
df.head()

,match_id,radiant_win,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,Radiant,Dire
0,5173127509,False,11,67,14,128,47,26,76,36,35,84,"R11,R67,R14,R128,R47","D26,D76,D36,D35,D84"
1,5173127511,False,74,84,21,41,83,35,93,47,128,45,"R74,R84,R21,R41,R83","D35,D93,D47,D128,D45"
2,5173127513,False,55,35,31,39,47,20,66,26,6,129,"R55,R35,R31,R39,R47","D20,D66,D26,D6,D129"
3,5173127515,True,4,47,86,5,99,111,16,121,82,93,"R4,R47,R86,R5,R99","D111,D16,D121,D82,D93"
4,5173127519,True,32,8,22,50,63,74,14,69,105,41,"R32,R8,R22,R50,R63","D74,D14,D69,D105,D41"


In [18]:
dfLogitRadiant = df['Radiant'].str.get_dummies(sep=',')
dfLogitDire = df['Dire'].str.get_dummies(sep=',')
dfLogit = dfLogitRadiant.join(dfLogitDire)
dfLogit['Radiant Win Y/N'] = df['radiant_win']
dfLogit.head()

,R1,R10,R100,R101,R102,R103,R104,R105,R106,R107,...,D91,D92,D93,D94,D95,D96,D97,D98,D99,Radiant Win Y/N
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,False
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,True
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True


This gave us a matrix where each row was a match and the X would represent a possible hero.
These dummy variables were needed because there are two groups of data, the Radiant and the Dire. This allows us to perform a single regression on our data.